In [ ]:
from ipywidgets import widgets
import numpy as np
import pandas as pd
import pulp


# CONSTANTS
PLANTS_TIERS = {'radiant': '金品质', 'flourishing': '紫品质',
                'hardy': '蓝品质', 'feeble': '白品质'}

PLANTS_I18N = {'fanged_geranium': '毒牙天竺葵', 'rose': '玫瑰',
               'puffapod': '泡泡豆荚', 'nifflers_fancy': '嗅幻草',
               'ladys_mantle': '羽衣草', 'mandrake': '曼德拉草',
               'chinese_chomping_cabbage': '中国咬人甘蓝',
               'peony': '牡丹', 'begonia': '秋海棠',
               'mayflower': '五月花', 'hydrangea': '绣球'}

In [ ]:
# 导入产品相关数据
df = pd.read_csv('plants.csv')

# 转换为 Categorical 类
df['species'] = pd.Categorical(df['species'])
df['tier'] = pd.Categorical(df['tier'])


# 去除含有 NaN 的行
df = df.dropna(subset=['gold'])

df = df.astype({'gold': int, 'gems': int, 'exp': float,
                'seed_price': int, 'harvest_time': int,
                'moisture': int, 'light': int,
                'emotional': int, 'pests': int})

# print(df.dtypes)
# df

In [ ]:
inventory_widgets = {}
for index, row in df.iterrows():
    inventory_widgets[f"{row['tier']}_{row['species']}"] = widgets.IntText(description=f"{PLANTS_TIERS[row['tier']]}_{PLANTS_I18N[row['species']]}",
                                                                           value=0,
                                                                           layout=widgets.Layout(width='400px'),
                                                                           style={'description_width': 'initial'},
                                                                           positiong='right')
    display(inventory_widgets[f"{row['tier']}_{row['species']}"])

In [ ]:
# 根据widgets更新植物库存
stocks = np.array([inventory_widgets[f'{row['tier']}_{row['species']}'].value for index, row in df.iterrows()])

# 植物名称列表
plants_cn = [f'{PLANTS_TIERS[row['tier']]}_{PLANTS_I18N[row['species']]}' for index, row in df.iterrows()]

# 植物单价
gold = np.array(df['gold'])

# 根据沙毕预算设定总价格
total_gold = 7500

# 创建问题实例，目标是最大化存货数量
prob = pulp.LpProblem("Maximize_Sale", pulp.LpMaximize)

# 决策变量，售出每种植物的件数，必须是整数
x = pulp.LpVariable.dicts("x", range(len(stocks)), lowBound=0, cat='Integer')

# 遍历，设置决策变量的上界为库存量
for i in range(len(stocks)):
    x[i].upBound = stocks[i]

# 目标函数：最大化总的存货数量
prob += pulp.lpSum([stocks[i] - x[i] for i in range(len(stocks))])

# 约束条件：每类产品售出数量乘以单价之和等于总价格
prob += pulp.lpSum([gold[i] * x[i] for i in range(len(stocks))]) == total_gold

# 求解问题
# CBC(Coin-or Branch and Cut)求解器使用分支定界算法来寻找整数规划问题的最优解。
solver = pulp.getSolver('PULP_CBC_CMD') 
prob.solve(solver=solver)

# 输出结果
if pulp.LpStatus[prob.status] == 'Optimal':
    print("Optimal solution found:")
    sold = 0
    for i, v in x.items():
        if v.varValue:
            if v.varValue > 0:
                print(f'{plants_cn[i]}: {int(v.varValue)}')
                sold += int(v.varValue) * gold[i]
    print("Total counts:", pulp.value(prob.objective))
    print("Total price:", sold)
else:
    print("No solution found.")

使用scipy的linprog，只能解决连续的线性规划问题，它不适用于整数规划问题。